## Station Names Mapping

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from sklearn.preprocessing import StandardScaler
import math
import operator
from geopy.distance import vincenty
import distance
%matplotlib inline

In [ ]:
bad_names = sta_traffic['Name'].apply(lambda x:str.lower(x))
good_names = sta['Name'].apply(lambda x:str.lower(x))
map_sta_names = {nam:{'bad_name':'','dist':99} for nam in good_names}

for good_name in map_sta_names.keys():
    for k in range(len(bad_names)):
        if int(distance.levenshtein(good_name, bad_names[k])) < map_sta_names[good_name]['dist']:
            map_sta_names[good_name]['dist'] = distance.levenshtein(good_name, bad_names[k])
            map_sta_names[good_name]['bad_name'] = bad_names[k]   

second_check = []
for good_name in map_sta_names.keys():
    if map_sta_names[good_name]['dist'] > 2:
        second_check.append((good_name, map_sta_names[good_name]['bad_name']))

not_matched = set(tup[0] for tup in second_check)

second_check_copy = [el for el in second_check]
for tup in second_check_copy:
    print(tup)
    testo = input('Is it a good matching?')
    if testo == 1:
        second_check.remove(tup)
    else:
        continue

map_sta_names_sure = {}
for good_name, dic in map_sta_names.iteritems():
    if good_name not in not_matched:
        map_sta_names_sure[good_name]=dic['bad_name']

import json
with open('map_sofar.json', 'w') as f:
    json.dump(map_sta_names_sure, f)

#Remove 'rd', 'th', 'st', '-', 'av', 'street', 'sq'
def stemit(sta_name):
    stem = sta_name.replace('rd', '').replace('th', '').replace('st','').replace('-', ' ').replace('av', '').replace('street', '').replace('sq', '')
    return stem

top_candidates = {tup[0]:{'bad_names':['','',''],'distz':[99,99,99]} for tup in second_check}

for good_name in top_candidates.keys():
    good_name_stem=stemit(good_name)
    for k in range(len(bad_names)):
        bad_name_stem=stemit(bad_names[k])
        if int(distance.levenshtein(good_name_stem, bad_name_stem)) < max(top_candidates[good_name]['distz']):
            idx = np.argmax(top_candidates[good_name]['distz'])
            top_candidates[good_name]['distz'][idx] = distance.levenshtein(good_name, bad_names[k])
            top_candidates[good_name]['bad_names'][idx] = bad_names[k]  

goods_to_remove = []
second_check_copy = [el for el in second_check]
for (good_name,bad_assigned) in second_check_copy:
    print(good_name)
    print(top_candidates[good_name]['bad_names'])
    bad_nam = raw_input("Which one?")
          
    if bad_nam != 'out':
          map_sta_names[good_name]['bad_name'] = bad_nam
          second_check.remove((good_name,bad_assigned))
    else: 
          goods_to_remove.append(good_name)

map_sta_names_sure = {}
for good_name, dic in map_sta_names.iteritems():
    if good_name not in goods_to_remove:
        map_sta_names_sure[good_name]=dic['bad_name']
        
import json
with open('map_sofar_fin.json', 'w') as f:
    json.dump(map_sta_names_sure, f)
    